<a href="https://colab.research.google.com/github/ozgursobe/btc_prediction/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3
import talib

In [ ]:
import talib
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
df = pd.read_csv("btc_data.csv")
df.head(4)
df = df.drop(["tradecount", "Volume USDT"], axis=1)
df.sort_values("Date", inplace=True)
df
df["target"] = (df['Close'].shift(-1) - df['Close'] > 0).astype(int)
df

EMA3 = talib.EMA(df["Close"], 3)
df["EMA3"] =EMA3
df["EMA3_UP"] =(df["Close"] >  EMA3).astype(int)

EMA6 = talib.EMA(df["Close"], 6)
df["EMA6"] = EMA6
df["EMA6_UP"] =(df["Close"] >  EMA6).astype(int)

EMA15 = talib.EMA(df["Close"], 15)
df["EMA15"] = EMA15
df["EMA15_UP"] =(df["Close"] >  EMA15).astype(int)

EMA30 = talib.EMA(df["Close"], 30)
df["EMA30"] = EMA30
df["EMA30_UP"] =(df["Close"] >  EMA30).astype(int)

MACD, MACD_SIGNAL, MACD_HIST = talib.MACD(df["Close"])
df["MACD"] = MACD
df["MACD_SIGNAL"] = MACD_SIGNAL
df["MACD_UP"] = (abs(MACD) > abs(MACD_SIGNAL)).astype(int)
df["MACD_HIST"] = MACD_HIST

RSI = talib.RSI(df["Close"])
STOCHRSI_FAST_K,STOCHRSI_FAST_D  = talib.STOCHRSI(df["Close"])
MOMENTUM = talib.MOM(df["Close"])
TRIMA = talib.TRIMA(df["Close"])

df["RSI"] = RSI
df["STOCHRSI_FAST_K"] = STOCHRSI_FAST_K
df["STOCHRSI_FAST_D"] = STOCHRSI_FAST_D
df["MOMENTUM"] = MOMENTUM
df["TRIMA"] = TRIMA
df["TRIMA_UP"] = (df["Close"] > TRIMA).astype(int)
df["RSI_Change"] = df["RSI"].diff()
df["MACD_HIST_Gradient"] = df["MACD_HIST"].diff()
df = df[33:]
df



In [ ]:
from sklearn.preprocessing import StandardScaler

features_to_scale = ["MACD_HIST","RSI","STOCHRSI_FAST_D","MOMENTUM"]
scaler = StandardScaler()

scaled_features = scaler.fit_transform(df[features_to_scale])
df[features_to_scale] = scaled_features

df = df[34:]

train = train = df[:-10000]
test = df[-10000:]





In [ ]:
# Logistic Regression Model
LOG_predictors = ["EMA3_UP", "MACD_UP", "MACD_HIST", "TRIMA_UP", "RSI_Change","MACD_HIST_Gradient"]
log = LogisticRegression()
model_LOG = log.fit(train[LOG_predictors], train["target"])

model_LOG.score(test[LOG_predictors], test["target"])

In [ ]:
# Random Forest Model
from sklearn.ensemble import RandomForestClassifier
FOREST_predictors = ["EMA3_UP","EMA6_UP", "MACD_UP", "MACD_HIST", "TRIMA_UP","RSI", "RSI_Change","MACD_HIST_Gradient","MOMENTUM", "EMA15_UP", "EMA30_UP","TRIMA"]
rforest = RandomForestClassifier(n_estimators=100,random_state=1, min_samples_split=5,max_depth=5)
model_forest = rforest.fit(train[FOREST_predictors], train["target"])
model_forest.score(test[FOREST_predictors], test["target"])


In [ ]:
# XGBoost
import xgboost as xgb
XGB_predictors = ["EMA3_UP", "MACD_HIST", "TRIMA_UP","RSI", "RSI_Change","MACD_HIST_Gradient","MOMENTUM"]
xgb = xgb.XGBClassifier(random_state = 1, max_depth =1, learning_rate=0.1, )
model_xgb = xgb.fit(train[XGB_predictors], train["target"])
model_xgb.score(test[XGB_predictors], test["target"])


In [ ]:
# Support Vector Classification Model
from sklearn.svm import SVC
SVM_predictors = ["EMA3_UP", "EMA6_UP", "TRIMA_UP", "RSI_Change",]
SVM = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=1)

model_SVM = SVM.fit(train[SVM_predictors], train["target"])
model_SVM.score(test[SVM_predictors], test["target"])


In [ ]:
# LSTM model
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Normalization
numerical_cols = ["EMA3_UP", "EMA6_UP", "TRIMA_UP", "RSI_Change","MACD_HIST_Gradient", "MACD_HIST", "MOMENTUM"]
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])



In [ ]:
import numpy as np

# Zaman penceresi boyutu (son N saat)
sequence_length = 60

# Giriş ve çıkış verilerini oluştur
X = []
y = []

for i in range(sequence_length, len(df)):
    X.append(df[numerical_cols].iloc[i-sequence_length:i].values)  # Son N saatin verisi
    y.append(df['target'].iloc[i])  # Bir sonraki günün hedefi

# X ve y'yi numpy array'e dönüştür
X = np.array(X)
y = np.array(y)

# Verinin boyutlarına bakalım
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Eğitim ve test verisine ayır
# Zaman serisi verilerinde sırayı bozmamak için shuffle=False kullanılır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Boyutları kontrol edelim
print(X_train)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Modeli oluştur
model = Sequential()

# LSTM katmanı
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))  # Dropout katmanı

# Bir başka LSTM katmanı
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))  # Dropout katmanı

# Çıkış katmanı
model.add(Dense(units=1, activation='sigmoid'))  # Sigmoid çünkü çıktı 0 veya 1 olacak

# Modeli derle
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Modelin özetini al
model.summary()



In [ ]:
# Modeli eğit
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Eğitimi görselleştirelim
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Test Doğruluğu')
plt.title('Model Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')
plt.legend()
plt.show()


In [ ]:
# Test doğruluğunu hesapla
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Kaybı: {test_loss}")
print(f"Test Doğruluğu: {test_accuracy}")


In [ ]:
# LSTM + CNN model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Flatten, Dropout, BatchNormalization

# Giriş boyutları
time_steps = 60  # Zaman adımları (örneğin 60 saat)
features = 7    # Özellik sayısı (örneğin RSI, EMA vb.)

# Giriş katmanı
input_layer = Input(shape=(time_steps, features))

# CNN Katmanı
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(input_layer)
conv1 = BatchNormalization()(conv1)
conv1 = Dropout(0.2)(conv1)

# LSTM Katmanı
lstm_layer = LSTM(50, return_sequences=False)(conv1)

# Yoğun Katman (Çıkış için)
dense1 = Dense(64, activation='relu')(lstm_layer)
dense1 = Dropout(0.3)(dense1)
output_layer = Dense(1, activation='sigmoid')(dense1)  # Binary classification için sigmoid

# Modeli birleştir
model = Model(inputs=input_layer, outputs=output_layer)

# Modeli derle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Özeti
model.summary()


In [ ]:
# Modeli eğit
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)


In [ ]:
# Doğrulama verisi performansı
val_loss, val_accuracy = model.evaluate(X_train, y_train)
print(f"Doğrulama Seti - Kayıp: {val_loss}, Doğruluk: {val_accuracy}")

# Test verisi performansı
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Seti - Kayıp: {test_loss}, Doğruluk: {test_accuracy}")


In [ ]:
import matplotlib.pyplot as plt

# Doğruluk grafiği
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.legend()
plt.title('Model Doğruluk Grafiği')
plt.show()

# Kayıp grafiği
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.legend()
plt.title('Model Kayıp Grafiği')
plt.show()
